In [2]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from data import ModelNet40
from model import PointNet, DGCNN_cls
import numpy as np
from torch.utils.data import DataLoader
from util import cal_loss, IOStream
import sklearn.metrics as metrics
import pickle

In [5]:
class args:
    def __init__(self):
        self.batch_size = 32
        self.num_points = 1024
        self.no_cuda = True
        self.seed = 1
        self.exp_name = "cls_1024_eval"
        self.model = "dgcnn"
        self.dataset = "modelnet40"
        self.test_batch_size = 16
        self.epochs = 250
        self.lr = 0.001
        self.momentum = 0.9
        self.scheduler = "cos"
        self.dropout = 0.5
        self.k = 20
        self.model_path = "pretrained/model.cls.1024.t7"
        self.emb_dims = 1024
args = args()

device = torch.device("cpu")

if not os.path.exists("outputs"):
    os.makedirs("outputs")
if not os.path.exists("outputs/" + args.exp_name):
    os.makedirs("outputs/" + args.exp_name)
if not os.path.exists("outputs/" + args.exp_name + "/" + "models"):
    os.makedirs("outputs/" + args.exp_name + "/" + "models")
os.system(
    "cp main_cls.py outputs" + "/" + args.exp_name + "/" + "main_cls.py.backup"
)
os.system("cp model.py outputs" + "/" + args.exp_name + "/" + "model.py.backup")
os.system("cp util.py outputs" + "/" + args.exp_name + "/" + "util.py.backup")
os.system("cp data.py outputs" + "/" + args.exp_name + "/" + "data.py.backup")

0

In [4]:
#sample test
test_loader = DataLoader(
        ModelNet40(partition="test", num_points=args.num_points),
        batch_size=args.test_batch_size,
        shuffle=True,
        drop_last=False,
    )


# Try to load models
if args.model == "pointnet":
    model = PointNet(args).to(device)
elif args.model == "dgcnn":
    model = DGCNN_cls(args).to(device)
else:
    raise Exception("Not implemented")

model = nn.DataParallel(model)
model.load_state_dict(torch.load(args.model_path, map_location=torch.device("cpu")))
model = model.eval()
test_acc = 0.0
count = 0.0
test_true = []
test_pred = []
for data, label in test_loader:
    count += 1
    print(count / len(test_loader))
    data, label = data.to(device), label.to(device).squeeze()
    data = data.permute(0, 2, 1)
    batch_size = data.size()[0]
    logits = model(data)
    preds = logits.max(dim=1)[1]
    test_true.append(label.cpu().numpy())
    test_pred.append(preds.detach().cpu().numpy())
test_true = np.concatenate(test_true)
test_pred = np.concatenate(test_pred)
test_acc = metrics.accuracy_score(test_true, test_pred)
avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
outstr = "Test :: test acc: %.6f, test avg acc: %.6f" % (
    test_acc,
    avg_per_class_acc,
)
print(outstr)

0.0064516129032258064
0.012903225806451613
0.01935483870967742


KeyboardInterrupt: 

In [13]:
#train
train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
    )
test_loader = DataLoader(
    ModelNet40(partition="test", num_points=args.num_points),
    num_workers=8,
    batch_size=args.test_batch_size,
    shuffle=True,
    drop_last=False,
)

device = torch.device("cpu")

# Try to load models
if args.model == "pointnet":
    model = PointNet(args).to(device)
elif args.model == "dgcnn":
    model = DGCNN_cls(args).to(device)
else:
    raise Exception("Not implemented")

print(str(model))

model = nn.DataParallel(model)
print("Let's use", torch.cuda.device_count(), "GPUs!")


print("Use Adam")
opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)

if args.scheduler == "cos":
    scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-3)
elif args.scheduler == "step":
    scheduler = StepLR(opt, step_size=20, gamma=0.7)

criterion = cal_loss
best_test_acc = 0

EPSILON = 0.5

for epoch in range(args.epochs):
    ####################
    # Train
    ####################
    train_loss = 0.0
    count = 0.0
    model.train()
    train_pred = []
    train_true = []
    for data, label in train_loader:
        data, label = data.to(device), label.to(device).squeeze()
        data = data.permute(0, 2, 1)
        # print(data.size())
        data.requires_grad = True
        batch_size = data.size()[0]
        opt.zero_grad()
        logits = model(data)
        loss = criterion(logits, label)
        loss.backward()

        #perform fgsm
        data_grad = data.grad.data
        perturbed = data + EPSILON * data_grad.sign()
        perturbed = torch.clamp(perturbed, 0, 1)
        perturbed_logits = model(perturbed)

        opt.step()
        preds = logits.max(dim=1)[1]
        preds_perturbed = perturbed_logits.max(dim=1)[1]
        print(preds)
        print(preds_perturbed)
        count += batch_size
        train_loss += loss.item() * batch_size
        train_true.append(label.cpu().numpy())
        train_pred.append(preds.detach().cpu().numpy())
        break
    break
    if args.scheduler == "cos":
        scheduler.step()
    elif args.scheduler == "step":
        if opt.param_groups[0]["lr"] > 1e-5:
            scheduler.step()
        if opt.param_groups[0]["lr"] < 1e-5:
            for param_group in opt.param_groups:
                param_group["lr"] = 1e-5
    train_true = np.concatenate(train_true)
    train_pred = np.concatenate(train_pred)
    outstr = "Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f" % (
        epoch,
        train_loss * 1.0 / count,
        metrics.accuracy_score(train_true, train_pred),
        metrics.balanced_accuracy_score(train_true, train_pred),
    )
    print(outstr)

DGCNN_cls(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (conv2): Sequential(
    (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias

In [6]:
mod = DGCNN_cls(args).to(device)
mod = nn.DataParallel(mod)
mod.load_state_dict(torch.load(args.model_path, map_location=torch.device("cpu")))

train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
    )
print("getting sample")
sample = None
label = None
for data, l in train_loader:
    print(0)
    data, l = data.to(device), l.to(device).squeeze()
    sample, label = data, l
    break


getting sample
0


In [10]:
sample[0]

tensor([[-0.0239,  0.1703, -0.1886],
        [ 0.2270,  0.1237, -0.2483],
        [ 0.2595,  0.1237, -0.4658],
        ...,
        [ 0.0851,  0.1819, -0.3375],
        [-0.4187,  0.1703, -0.5937],
        [ 0.0424,  0.0821,  0.9305]])